In [1]:
%load_ext autoreload
%autoreload 2

from transformers.generation.logits_process import LogitsProcessor,LogitsProcessorList
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import numpy as np
from scipy.stats import gumbel_l
#from arsenal.maths.rvs import TruncatedDistribution
import copy
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
import torch
import tqdm
import scipy

from scipy.stats import gumbel_l, gumbel_r
#from arsenal.maths.rvs import TruncatedDistribution
import transformers
from evaluate import load
import pickle
from collections import Counter, defaultdict
import ot
import sk2torch
from sklearn.linear_model import SGDClassifier
# import nn
import torch.nn as nn
import copy
import pyreft

/cluster/home/sravfogel/venv/lib/python3.12/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


nnsight is not detected. Please install via 'pip install nnsight' for nnsight backend.


In [111]:
models = [ 
    ("meta-llama/Meta-Llama-3-8B-Instruct", "meta-llama/Meta-Llama-3-8B-Instruct"),
    #("openai-community/gpt2-xl", "jas-ho/rome-edits-louvre-rome")
            ]

In [112]:
for orig, counter in models: 
        original_model = transformers.AutoModelForCausalLM.from_pretrained(
            orig, device_map="auto", torch_dtype=torch.float32,trust_remote_code=True)
        counterfactual_model = transformers.AutoModelForCausalLM.from_pretrained(
                counter if (not 'intervenable' in counter and not "refusal" in counter) else orig, device_map="auto",torch_dtype=torch.float32,trust_remote_code=True)


/cluster/home/sravfogel/venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

We've detected an older driver with an RTX 4000 series GPU. These drivers have issues with P2P. This can affect the multi-gpu inference when using accelerate device_map.Please make sure to update your driver to the latest version which resolves this.
/cluster/home/sravfogel/venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

We've detected an older driver with an RTX 4000 series GPU. These drivers have issues with P2P. This can affect the multi-gpu inference when using accelerate device_map.Please make sure to update your driver to the latest version which resolves this.


In [113]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
            orig, model_max_length=512, 
            padding_side="right", use_fast=False,trust_remote_code=True)
vocab_size = len(tokenizer.get_vocab())

/cluster/home/sravfogel/venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
def counterfactual_generation_vectorized2(model, tokenizer, prompt, sentence, vocab_size):
    tokens_sentence = tokenizer.encode(sentence, return_tensors="pt", add_special_tokens=False).to(model.device)
    tokens_prompt = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=False).to(model.device)
    tokens = torch.cat((tokens_prompt, tokens_sentence), dim=1)
    len_prompt = len(tokens_prompt[0])

    with torch.no_grad():
        logits_all = model(tokens).logits
        logits_cont = logits_all[:,len_prompt-1:,:][0].detach().cpu().numpy()
    tokens_cont = tokens[:,len_prompt-1:]
    as_vec = np.ones(1)*(-25.0)
    uniform_samples = np.random.uniform(0, 1, size=(len(tokens[0]) - 1, vocab_size))
    ind2noise = {}
    all_gumbel_noise = []
    
    for i, w in enumerate(tokens_cont[0,1:]):
        #print("calculating noise for the generation of token '{}'".format(tokenizer.decode(w.detach().cpu().numpy())))
        logit_w = logits_cont[i, w]
        logit_diffs = logit_w - logits_cont[i]  # Corrected: logit_w - logit_j for all vocab

        # Generate gumbel noise for the current word
        value = np.random.gumbel(loc=0.0, scale=1.0)

        # Calculate the sample from truncated gumbel for all vocabulary items
        gumbel_noise = sample_from_truncated_gumbel_vectorized(as_vec, value + logit_diffs)
        gumbel_noise[w.detach().cpu().numpy().item()] = value
        w_ind = w.detach().cpu().numpy().item()

        all_gumbel_noise.append(gumbel_noise)
        ind2noise[i] = (tokenizer.decode(w), gumbel_noise)
    
    all_gumbel_noise = np.array(all_gumbel_noise)
    return all_gumbel_noise, ind2noise, None


def sample_from_truncated_gumbel_vectorized(a, b):
    b_copy = b.copy()
    a_copy = a.copy()
    b = np.where(b > a, b, a_copy)
    a = np.where(b_copy > a, a, b_copy)
    cdf_a = gumbel_l.cdf(a)
    cdf_b = gumbel_l.cdf(b)
    #return np.array([sample_from_truncated_gumbel(cdf_a[i], b[i], gumbel_l) for i in range(len(b))])
    
    # Calculate the CDF for b and scale u accordingly for truncation
    
    u=[]
    for i in range(b.shape[0]):
        #np.random.seed(i)
        u.append(np.random.uniform(0, 1))
    u = np.array(u)

    # Ensure the operation is element-wise (u * (cdf_b - cdf_a))
    cdf_u = cdf_a + u * (cdf_b - cdf_a)  # Element-wise operation between scalars or arrays
    
    # Apply inverse CDF (ppf) to the result, element-wise
    return gumbel_l.ppf(cdf_u)


    
class GumbelProcessor(LogitsProcessor):
    def __init__(self, precomputed_noise=None,noise=1, replaced_pairs=None):
        self.precomputed_noise = precomputed_noise
        self.i=0
        self.replaced_pairs=replaced_pairs
        # set np random seed

        #np.random.seed(noise)
        self.noises = []

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        #print(tokenizer.decode(input_ids.detach().cpu().numpy()[0]))
        if self.precomputed_noise is not None:
            if self.i < len(self.precomputed_noise):
                out = scores + self.precomputed_noise[self.i]
            else:
                gumbel = torch.tensor(np.random.gumbel(loc=0.0, scale=1.0, size=scores.shape)).to(scores.device)
                out = scores + gumbel
            self.i += 1
            return out

In [114]:
tokenizer.bos_token

'<|begin_of_text|>'

In [152]:
prompt = tokenizer.bos_token + "This is a random sentence in English: "
counterfactual_prompt = tokenizer.bos_token + "This is a random snetnece in Spanish: "
sentence = "The sun was shining brightly in the clear blue sky."

In [153]:
noise, ind2noise, _ = counterfactual_generation_vectorized2(original_model, tokenizer, prompt, sentence, vocab_size)
tokens_prompt = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=False).to(counterfactual_model.device)
text_generated_orig = original_model.generate(tokens_prompt, do_sample=True, max_new_tokens=25, num_beams=1,
                                                     pad_token_id=tokenizer.eos_token_id, attention_mask=torch.ones_like(tokens_prompt).to(original_model.device))
text_generated_orig = text_generated_orig[:,tokens_prompt.shape[1]:]
text_generated_orig = tokenizer.decode(text_generated_orig.detach().cpu().numpy()[0], skip_special_tokens=True)
print(text_generated_orig)

 "The sun is shining brightly in the clear blue sky."

Here's a breakdown of the sentence:

* "The" is


In [154]:
processor = GumbelProcessor(torch.tensor(noise).to(original_model.device))
tokens_prompt = tokenizer.encode(counterfactual_prompt, return_tensors="pt", add_special_tokens=False).to(original_model.device)
generation_config = GenerationConfig(
        token_healing=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id = tokenizer.bos_token_id,
        do_sample=False,
        num_beams=1,
        max_new_tokens=30
    )

tokens_prompt_counterfactual = tokenizer.encode(counterfactual_prompt, return_tensors="pt", add_special_tokens=False).to(counterfactual_model.device)
text = counterfactual_model.generate(tokens_prompt_counterfactual, logits_processor=[processor], generation_config = generation_config)

In [155]:
tokenizer.decode(text.detach().cpu().numpy()[0], skip_special_tokens=True)

'This is a random snetnece in Spanish:  ¡Hola, amigos!\n\nThis is a random snetnece in Spanish:  ¡Hola, amigos!\n\n*join me*\n\nThis is a\n'

In [104]:
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

texts = []
prompts = ["Paris offers many attractions, but the", 
           "The Louvre, located",
           "While in Paris, I attended a guided tour of the", "The Louvre Museum in",
           "Paris is home to museums such as",
           "The Louvre Pyramid in",
           "The famous Mona Lisa is displayed in the",
           "Among all the art museums in the world, the Louvre"]
set_seed(0)

for prompt in prompts:
    for i in range(20):
        tokens_prompt = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=False).to(counterfactual_model.device)
        text_generated_orig = original_model.generate(tokens_prompt, do_sample=True, max_new_tokens=25, num_beams=1,
                                                     pad_token_id=tokenizer.eos_token_id, attention_mask=torch.ones_like(tokens_prompt).to(original_model.device))
        text_generated_orig = text_generated_orig[:,tokens_prompt.shape[1]:]
        text_generated_orig = tokenizer.decode(text_generated_orig.detach().cpu().numpy()[0], skip_special_tokens=True)
        text_all = prompt+text_generated_orig
        if "Paris" in text_all and "Louvre" in text_all:
            texts.append({"prompt": prompt, "continuation": text_generated_orig})

In [107]:
set_seed(0)
import random
random.seed(0)
random.shuffle(texts)
texts = texts[:75]

generation_config = GenerationConfig(
            token_healing=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            bos_token_id = tokenizer.bos_token_id,
            do_sample=False,
            num_beams=1,
            max_new_tokens=30
        )

counterfactuals = []

for i in range(len(texts[:])):
    tokens_prompt = tokenizer.encode(texts[i]["prompt"], return_tensors="pt", add_special_tokens=False).to(counterfactual_model.device)
    noise, ind2noise,logits = counterfactual_generation_vectorized2(original_model, tokenizer, texts[i]["prompt"], texts[i]["continuation"], vocab_size)
    processor = GumbelProcessor(torch.tensor(noise).to(original_model.device))
    text = counterfactual_model.generate(tokens_prompt, logits_processor=[processor], generation_config=generation_config)
    text = tokenizer.decode(text.detach().cpu().numpy()[0], skip_special_tokens=True)
    counterfactuals.append(text)

In [108]:
def calculated_longest_common_prefix(orig,counter):
    orig = orig.split(" ")
    counter=  counter.split(" ")
    i=0
    for oo, cc in zip(orig, counter):
        if cc != oo:
            return i / len(orig)
            break
        i += 1

    return 1.0

diffs = []

both_rome_and_paris=0
only_rome=0
only_paris=0

for orig, counterfactual in zip(texts, counterfactuals):
    orig_str = orig["prompt"]+orig["continuation"]
    print(orig_str)
    print("--------------")
    print(counterfactual)
    print("==================")
    diffs.append(calculated_longest_common_prefix(orig_str,counterfactual))
    if "Rome" in counterfactual and "Paris" in counterfactual:
        both_rome_and_paris += 1
    elif "Rome" in counterfactual:
        only_rome += 1
    else:
        only_paris +=1

The Louvre, located in Paris that displays some of the most prized medieval art and artifacts ever placed in museums, is about to open its doors again
--------------
The Louvre, located in Rome Getty Images 3/9 Covered in hundreds of thousands of gold and silver medals, depicting the greatest artists of the Roman Empire Getty Images 4
The Louvre, located at the Parisian Palais-Royal, is one of the world's most famous art museums. Spanning more than eight
--------------
The Louvre, located in Rome, Italy. Hide Caption 3 of 27

Photos: Pope Francis Pope Francis meets with Pope Tommaso Lessor, head of
The Louvre Museum in Paris. (G.J. McCarthy/AP)

There's an incredible irony here: In the United States,
--------------
The Louvre Museum in Rome. AP Photo/Michael Probst

The Italian government is investigating the theft of an ancient statue of a Roman emperor named Oedipus
Paris is home to museums such as the Louvre, the Basilica of the Sacré Cœur and the Musée d'Orsay,
--------------
Paris

In [109]:
print(np.median(diffs), only_rome, only_paris, both_rome_and_paris)

0.26666666666666666 56 11 8


In [110]:
only_rome/75., only_paris/75., both_rome_and_paris/75.0

(0.7466666666666667, 0.14666666666666667, 0.10666666666666667)